In [11]:
import pandas as pd
import numpy as np
import os
%pprint off

Pretty printing has been turned ON


In [12]:
fpath = '/Users/songheekim/GoogleDrive/Primary/Projects/VerbVector/AMT/html/experiments/ratings2/results/results_clean_0628.csv'

In [24]:
res = pd.read_csv(fpath)
res = res.sort_values(by=['stim_id', 'id'])

## check number of responses per lemma
# for i in set(res['stim_id']): 
#     df = res.loc[res['stim_id']==i]
#     print (i, df.shape[0])

In [25]:
res

,id,stim_id,stim,subject_id,response,start,end,rt,buttonpress,turkcode,subject_num,gender,age,education,occupation,results_id,catch_subject_id,catch_response,correctN,badsubject
213,214,1,live,206,6_4_1_3_2_6_2_6_6_5_1_4_6_6_4_2_1_1_2_1_1_5_5_...,2021-05-26 11:38:39,2021-05-26 11:46:03,2253_6104_4009_3299_5748_9628_5439_2842_2449_3...,1_1_2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_...,4940126,A1ZDR2UN7M92LI,m,37,16.0,Foodprep,214,206,0_6_0_6_6,4,N
220,221,1,live,213,3_0_0_0_0_1_0_1_4_2_3_2_2_5_3_0_0_0_0_0_0_3_3_...,2021-05-26 11:35:37,2021-05-26 11:49:09,8232_1864_2304_1760_3624_3744_2015_2841_2776_4...,1_1_1_1_1_1_1_1_1_1_1_2_1_1_1_1_1_1_1_1_1_1_1_...,4019101,ALOYY3E72AQ1K,f,56,16.0,digitalproductmanagerworkingononlinecontentfor...,221,213,0_6_0_6_6,4,N
231,232,1,live,223,2_2_0_2_1_0_0_5_5_3_1_3_4_5_3_1_0_0_1_0_0_0_4_...,2021-05-26 10:54:50,2021-05-26 11:53:10,19852_7922_172288_4293_73124_2751_118019_21132...,1_1_1_2_1_1_1_1_2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,9110817,A171S8E9IFSHH2,m,33,14.0,healthcaresupport,232,223,0_6_0_6_6,4,N
840,841,1,live,286,0_0_0_0_0_0_1_0_5_0_3_2_1_3_1_0_0_0_0_0_0_1_1_...,2021-05-30 07:16:46,2021-05-30 07:26:46,14411_3015_1406_5198_1980_2520_4139_28823_8707...,1_1_1_2_1_1_2_2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,4202983,A1EJ4HA4K85PBJ,f,53,18.0,communicationsdirectorforagovernmentagency,287,286,0_6_0_6_6,4,N
1095,1096,1,live,266,2_1_0_0_7_0_0_4_6_2_1_0_4_6_2_1_0_0_1_0_0_2_3_...,2021-06-03 17:58:11,2021-06-03 18:06:46,12905_5097_2779_3240_5073_3374_2363_4421_3701_...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,8140924,A1AZLJ3R5FQ9B7,f,67,16.0,Selfemployedwriter,542,266,0_6_0_6_6,4,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,1229,320,overwhelm,197,3_0_0_0_1_4_0_2_1_1_5_2_1_1_1_0_0_0_0_7_3_5_1_...,2021-06-04 04:18:12,2021-06-04 04:35:58,2677_4588_2250_4185_2925_11564_2398_4005_3430_...,1_1_1_1_2_1_1_1_1_1_1_1_1_1_2_1_1_1_1_1_1_3_2_...,8480686,A3UWHJTAUTF6TO,o,24,16.0,financemanager,675,197,0_6_0_6_6,4,N
1699,1700,320,overwhelm,85,0_0_0_0_0_3_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_...,2021-06-14 12:07:41,2021-06-14 12:11:07,2097_1664_1768_1825_1831_1904_1904_1712_2008_1...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,3507655,A3DUPRZSMU9W5R,f,45,16.0,customerservicerep,1146,85,0_6_0_6_6,4,N
2043,2044,320,overwhelm,12,7_4_7_0_7_2_7_7_7_2_0_7_7_7_7_7_7_7_7_7_1_7_7_...,2021-06-15 16:10:53,2021-06-15 16:35:05,8708_5163_2199_3712_56350_937459_3624_2469_316...,3_1_1_2_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_1_1_...,4478957,A2Z6EQWA4XJQGI,f,26,16.0,NaN,1490,12,0_6_0_6_6,4,N
2342,2343,320,overwhelm,151,1_0_0_0_0_0_0_2_1_0_0_0_1_1_0_0_0_0_0_0_0_1_2_...,2021-06-21 14:52:34,2021-06-21 14:57:50,2296_1850_3049_1850_2221_3117_3440_2134_2112_4...,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_1_...,5440899,AKSLU0C30G3JT,m,48,16.0,Director,1789,151,0_6_0_6_6,4,N


In [ ]:
##### descriptive statistics on RT, subject, 
####  intersubject correlation per lemma
####  sonify
####  klustering